<a href="https://colab.research.google.com/github/aksanachmel/DFDEDATA6_EX2/blob/main/IBM_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#          Hands-on Lab: Access REST APIs & Request HTTP

In [ ]:
# An API lets two pieces of software talk to each other. Just like a function, you don’t have to know how the API works only its inputs and outputs. 
#An essential type of API is a REST API that allows you to access resources via the internet. 
#In this lab, we will review the Pandas Library in the context of an API, we will also review a basic REST API.

In [1]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance

     |████████████████████████████████| 70 kB 2.9 MB/s 


Pandas is actually set of software components , much of which is not even written in Python.

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

You create a dictionary, this is just data.

In [3]:
dict_={'a':[11,21,31],'b':[12,22,32]}

When you create a Pandas object with the Dataframe constructor in API lingo, this is an "instance". The data in the dictionary is passed along to the pandas API. You then use the dataframe to communicate with the API.

In [4]:
df=pd.DataFrame(dict_)
type(df)

pandas.core.frame.DataFrame

In [5]:
df.head()

,a,b
0,11,12
1,21,22
2,31,32


In [6]:
df.mean()

a    21.0
b    22.0
dtype: float64

REST APIs
Rest API’s function by sending a request, the request is communicated via HTTP message. The HTTP message usually contains a JSON file. This contains instructions for what operation we would like the service or resource to perform. In a similar manner, API returns a response, via an HTTP message, this response is usually contained within a JSON.

In cryptocurrency a popular method to display the movements of the price of a currency.



Lets start off by getting the data we need. Using the get_coin_market_chart_by_id(id, vs_currency, days). id is the name of the coin you want, vs_currency is the currency you want the price in, and days is how many days back from today you want.

In [8]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)

In [ ]:
type(bitcoin_data )

The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.

In [11]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]

[[1646517720686, 39453.91956524555],
 [1646521320617, 39515.00049382657],
 [1646524861617, 39465.92350920515],
 [1646528476884, 39608.44670346521],
 [1646532136922, 39522.024591525966]]

Finally lets turn this data into a Pandas DataFrame.

In [12]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])

Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called Date. We will map our unix_to_datetime to each timestamp and convert it to a readable datetime.

In [13]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

Using this modified dataset we can now group by the Date and find the min, max, open, and close for the candlesticks.

In [14]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

Finally we are now ready to use plotly to create our Candlestick Chart.

In [15]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()